### No need of the patches (from all the tissue area) split by CLAM
### Need the correspondent WSI instead
### For multi-thread, better to use Spyder (also seems faster) than Jupyter. Jupyter will stop all the graphic views (e.g. webpages) if exceed memory.

In [1]:
import os
import re
from PIL import Image
import h5py
import numpy as np
import math
# import sys
# !conda install --yes --prefix {sys.prefix} ipywidgets # install in the current conda environment
# !jupyter nbextension enable --py widgetsnbextension 
# Then restart jupyter
from tqdm.notebook import tqdm
## for Spyder
#from functools import partial
#tqdm = partial(tqdm, position=0, leave=True)
# from numba import jit, njit, cuda
import openslide

In [2]:
path_tiles_qp = "/media/visiopharm5/WDRed(backup)/qupath_mondor/tiles_20x_256"
path_slide = "/media/visiopharm5/WDGold/deeplearning/Hepatocarcinomes/slides_annotations_hammamatsu"
path_patch_anno = "/media/visiopharm5/WDRed(backup)/clam_extension/results/patches_mondor_tumor"

In [3]:
# Iterate the whole directory
slide_id = []
tile_id = []
mask_id =  []

start = 0
stop = 10 

for root, dirs, files in os.walk(path_tiles_qp):
#     display(dirs)
    slide_id =  sorted(dirs)[start:stop] # sort the dirs to ensure the order, then for indexing
    break
print(slide_id)
        
for name in slide_id:
    tmp_tile = []
    tmp_mask = []
    for root, dirs, files in os.walk(os.path.join(path_tiles_qp, name)):
        if len(files) != 0:
            for file in files:
                if file.endswith(".tif"):
                    tmp_tile.append(file)
                elif file.endswith(".png"):
                    tmp_mask.append(file)
            tile_id.append(tmp_tile)
            mask_id.append(tmp_mask)
        
for i in range(len(slide_id)):
    if tile_id[i][0].startswith(slide_id[i]) and mask_id[i][0].startswith(slide_id[i]):
        pass
    else:
        raise Exception("os.walk() returns arbitrary orders.")

['HMNT0071 - 2017-06-15 11.35.00', 'HMNT0071_bis - 2017-06-15 12.25.18', 'HMNT0073_bis - 2017-06-17 22.30.07', 'HMNT0100 - 2017-07-14 07.59.28', 'HMNT0106 - 2017-06-11 09.45.50', 'HMNT0106_bis - 2017-06-11 10.39.38', 'HMNT0109 - 2017-06-18 16.13.47', 'HMNT0109_bis - 2017-06-18 17.10.12', 'HMNT0110 - 2017-06-16 21.50.50', 'HMNT0116 - 2017-06-11 13.05.47']


In [4]:
# ## Specific list to process
# list_proc = ["TCGA-G3-AAV6-01Z-00-DX1.F4D85E6A-7F71-47D4-BEB6-984FDD218A9F"]
# slide_id = []
# tile_id = []
# mask_id =  []
# for root, dirs, files in os.walk(path_tiles_qp):
#     for name in dirs:
#         if name in list_proc:
#             slide_id.append(name)
#     tmp_tile = []
#     tmp_mask = []
#     if root.split("/")[-1] in list_proc:
#         for file in files:
#             if file.endswith(".tif"):
#                 tmp_tile.append(file)
#             elif file.endswith(".png"):
#                 tmp_mask.append(file)
#         tile_id.append(tmp_tile)
#         mask_id.append(tmp_mask)

In [5]:
# print(slide_id[9])
display(slide_id)

['HMNT0071 - 2017-06-15 11.35.00',
 'HMNT0071_bis - 2017-06-15 12.25.18',
 'HMNT0073_bis - 2017-06-17 22.30.07',
 'HMNT0100 - 2017-07-14 07.59.28',
 'HMNT0106 - 2017-06-11 09.45.50',
 'HMNT0106_bis - 2017-06-11 10.39.38',
 'HMNT0109 - 2017-06-18 16.13.47',
 'HMNT0109_bis - 2017-06-18 17.10.12',
 'HMNT0110 - 2017-06-16 21.50.50',
 'HMNT0116 - 2017-06-11 13.05.47']

In [6]:
for i in range(len(slide_id)):
    print(tile_id[i][0])

HMNT0071 - 2017-06-15 11.35.00 [x=43008,y=6656,w=512,h=512].tif
HMNT0071_bis - 2017-06-15 12.25.18 [x=48128,y=11264,w=512,h=512].tif
HMNT0073_bis - 2017-06-17 22.30.07 [x=38400,y=10752,w=512,h=512].tif
HMNT0100 - 2017-07-14 07.59.28 [x=65536,y=4096,w=512,h=512].tif
HMNT0106 - 2017-06-11 09.45.50 [x=70656,y=27648,w=512,h=512].tif
HMNT0106_bis - 2017-06-11 10.39.38 [x=33792,y=12800,w=512,h=512].tif
HMNT0109 - 2017-06-18 16.13.47 [x=59392,y=5120,w=512,h=512].tif
HMNT0109_bis - 2017-06-18 17.10.12 [x=61952,y=15872,w=512,h=512].tif
HMNT0110 - 2017-06-16 21.50.50 [x=49664,y=6144,w=512,h=512].tif
HMNT0116 - 2017-06-11 13.05.47 [x=46080,y=12288,w=512,h=512].tif


In [7]:
# re.split("=|,",'TCGA-G3-AAV6-01Z-00-DX1.F4D85E6A-7F71-47D4-BEB6-984FDD218A9F [x=10240,y=19968,w=512,h=512].tif')

In [8]:
# print(mask_id[0:3])

In [9]:
def main(s:int=0):
    for i in tqdm(range(s, len(tile_id))): # for each slide / bag
        print("\n"+str(i)+": "+slide_id[i])
        if os.path.isfile(os.path.join(path_patch_anno, slide_id[i]+'.h5')):
            print("Already existed. Source folder is" + path_tiles_qp.split("/")[-1] + ".")
        else:
            with h5py.File(os.path.join(path_patch_anno, slide_id[i]+'.h5'), "w") as f: # create a h5 file for each bag
                init = True

                for j in tqdm(range(len(tile_id[i]))): # for each patch
                    mask = np.asarray(Image.open(os.path.join(path_tiles_qp, slide_id[i], tile_id[i][j]
                                                              .replace(".tif","-labelled.png"))))
                    if sum(sum(mask==1)) >= 32768: # only annotated higher than 50% will be considered
                        # coords
                        x = int(re.split("=|,",tile_id[i][j])[1])//2
                        y = int(re.split("=|,",tile_id[i][j])[3])//2
                        if init:
                            coords = [x, y]
                        else:
                            coords = np.vstack((coords, [x, y]))

                        # imgs
                        if init:
                            imgs = np.asarray(Image.open(os.path.join(path_tiles_qp, slide_id[i], tile_id[i][j])))
                            imgs = imgs[np.newaxis, :, :, :]
                            init = False
                        else:
                            imgs = np.vstack((imgs, np.asarray(Image.open(os.path.join(path_tiles_qp, slide_id[i], 
                                                                                       tile_id[i][j])))[np.newaxis, :, :, :]))
                coords = coords.astype(np.int32) # convert from string to int32
                f.create_dataset("coords", data=coords)
                imgs = f.create_dataset("imgs", data=imgs)
                
                with openslide.OpenSlide(os.path.join(path_slide, slide_id[i]+".ndpi")) as f_ref:
                    # f_ref.properties['aperio.AppMag']
                    if (f_ref.properties[openslide.PROPERTY_NAME_OBJECTIVE_POWER] == '20'): # 20x  ~= 0.5 or 1.0 (not correctly recognized)
                        downsample = 1.0
                    elif (f_ref.properties[openslide.PROPERTY_NAME_OBJECTIVE_POWER] == '40'): # 40x pixelsize ~= 0.25
                        downsample = 2.0
                    else:
                        raise Exception("The highest magnification should be 20x or 40x.")
                        
                    # coords have no attributes
                    imgs.attrs["downsample"] = np.asarray([np.float64(1), np.float(1)])
                    imgs.attrs["downsampled_level_dim"] = np.asarray([np.int64(math.floor(f_ref.dimensions[0]/downsample)), np.int64(math.floor(f_ref.dimensions[1]/downsample))])
                    imgs.attrs["level_dim"] = np.asarray(f_ref.dimensions)
                    imgs.attrs["patch_level"] = np.int64(0)
                    imgs.attrs["wsi_name"] = slide_id[i]


In [ ]:
main(0)


0: HMNT0071 - 2017-06-15 11.35.00
